In [11]:
import pandas as pd
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
from torch.utils.data import DataLoader
from torch.utils.data import Dataset 


In [12]:
#生成训练集
class Train_Loader(Dataset):
    def __init__(
            self, 
            data_dir, 
    ):
        self.data_dir = data_dir
        data = pd.read_excel(self.data_dir, skiprows=0)
        #将数据类型转换为float
        data = data.astype('float32')
        #缺失值使用上一行和下一行同一列的平均值填充
        data = data.fillna(data.mean())
        #对目标列(除最后三列)归一化
        scaler = MinMaxScaler(feature_range=(0, 1))
        data.iloc[:, :-3] = scaler.fit_transform(data.iloc[:, :-3])
        self.data = data.values
        self.scaler = scaler

    def __getitem__(self, i):
        features = self.data[i, :-3]
        #扩充为二维
        features = np.expand_dims(features, axis=0)
        targets = self.data[i, -3:]
        #扩充为二维
        targets = np.expand_dims(targets, axis=0)
        return features, targets
        
    def __len__(self):
        return len(self.data)
    

#生成测试集
class Test_Loader(Dataset):
    def __init__(
            self, 
            data_dir, 
            scaler,
    ):
        self.data_dir = data_dir
        data = pd.read_excel(self.data_dir, skiprows=0)
        #将数据类型转换为float
        data = data.astype('float32')
        #缺失值使用上一行和下一行同一列的平均值填充
        data = data.fillna(data.mean())
        #对目标列(除最后三列)归一化
        data.iloc[:, :-3] = scaler.transform(data.iloc[:, :-3])
        self.data = data.values

    def __getitem__(self, i):
        features = self.data[i, :-3]
        #扩充为二维
        features = np.expand_dims(features, axis=0)
        targets = self.data[i, -3:]
        #扩充为二维
        targets = np.expand_dims(targets, axis=0)
        return features, targets
        
    def __len__(self):
        return len(self.data)

In [13]:
train_data_dir="data/train_data.xlsx"
train_data_loader = Train_Loader(train_data_dir)

In [14]:
#查看feature，target数据形状
features, targets = train_data_loader[0]
print(features.shape)
print(targets.shape)
#查看train_data_loader数据长度
print(len(train_data_loader))

(1, 65)
(1, 3)
20157


In [15]:
batchsize=64
#创建数据加载器
train_data = DataLoader(train_data_loader, batch_size=batchsize, shuffle=True)
#查看数据加载器，数据形状
for features, targets in train_data:
    print(features.shape)
    print(targets.shape)
    break
print(len(train_data))

torch.Size([64, 1, 65])
torch.Size([64, 1, 3])
315


In [16]:
from nets.CNN_BIGRU_Attention import CNN_BiGRU_Attention
n_future = 65  # Example value, adjust as needed
n_class = 3   # Example value, adjust as needed
model = CNN_BiGRU_Attention(n_future, n_class)

#定义损失函数
criterion = nn.MSELoss()
#定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)
model_dir="model/CNN_BiGRU_Attention"



In [17]:
# model=torch.load("model/CNN_BiGRU_Attention588.952.pth")


In [18]:
#计算验证集的MSE
valid_data_dir="data/valid_data.xlsx"
valid_data_loader = Test_Loader(valid_data_dir, train_data_loader.scaler)
valid_data = DataLoader(valid_data_loader, batch_size=batchsize, shuffle=False)
print(len(valid_data))


79


In [19]:
def valid(valid_model, valid_data, criterion):
    # valid_model.eval()
    valid_loss = 0
    for features, targets in valid_data:
        features = features.cuda()
        targets = targets.cuda()
        valid_model = valid_model.cuda()
        outputs = valid_model(features)
        loss = criterion(outputs, targets)
        valid_loss = valid_loss+loss.item()
    # print('valid_loss:', valid_loss / len(valid_data))
    return valid_loss / len(valid_data)

In [20]:
#直接训练模型，根据验证集MSE保存最优模型，超过50伦验证集MSE没有下降则修改学习率
min_valid_loss = float('inf')
no_improve = 0
for epoch in range(1000):
    model.train()
    train_loss = 0
    for features, targets in train_data:
        features = features.cuda()
        targets = targets.cuda()
        optimizer.zero_grad()
        model = model.cuda()
        outputs = model(features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss = train_loss+loss.item()
    valid_loss = valid(model, valid_data, criterion)
    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        torch.save(model, model_dir + str(min_valid_loss) + '.pth')
        no_improve = 0
    else:
        no_improve += 1
    if no_improve > 50:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
            print('Learning rate has been changed to: {}'.format(param_group['lr']))
        no_improve = 0
    print('Epoch: {}, Train Loss: {:.4f}, Valid Loss: {:.4f}'.format(epoch, train_loss / len(train_data), valid_loss))
    if optimizer.param_groups[0]['lr'] < 1e-6:
        break


Epoch: 0, Train Loss: 50290.9064, Valid Loss: 13870.2512
Epoch: 1, Train Loss: 12295.0644, Valid Loss: 12102.0718
Epoch: 2, Train Loss: 5175.6430, Valid Loss: 719.7935
Epoch: 3, Train Loss: 583.6627, Valid Loss: 497.3935
Epoch: 4, Train Loss: 429.0909, Valid Loss: 388.9074
Epoch: 5, Train Loss: 384.9320, Valid Loss: 370.0553
Epoch: 6, Train Loss: 371.1830, Valid Loss: 362.8242
Epoch: 7, Train Loss: 362.4310, Valid Loss: 349.6757
Epoch: 8, Train Loss: 353.5115, Valid Loss: 337.1813
Epoch: 9, Train Loss: 338.5733, Valid Loss: 319.3036
Epoch: 10, Train Loss: 308.0131, Valid Loss: 273.6759
Epoch: 11, Train Loss: 255.4690, Valid Loss: 220.2356
Epoch: 12, Train Loss: 203.7031, Valid Loss: 179.2967
Epoch: 13, Train Loss: 178.9815, Valid Loss: 164.1949
Epoch: 14, Train Loss: 164.9173, Valid Loss: 157.4228
Epoch: 15, Train Loss: 148.2069, Valid Loss: 139.7091
Epoch: 16, Train Loss: 136.5460, Valid Loss: 139.0116
Epoch: 17, Train Loss: 128.4460, Valid Loss: 129.5499
Epoch: 18, Train Loss: 125.63

In [22]:
# model=torch.load("model/CNN_Transformer240.1402118945944.pth")
model=torch.load("model/CNN_BiGRU_Attention52.89754729934886.pth")
#计算测试集MSE
test_data_dir="data/test_data.xlsx"
test_data_loader = Test_Loader(test_data_dir, train_data_loader.scaler)
test_data = DataLoader(test_data_loader, batch_size=1, shuffle=False)
print(len(test_data))

4447


In [23]:
#计算测试集上的MSE,RMSE,MAE,R2
def test(test_model, test_data):
    test_model.eval()

    YS_true = []
    YS_pred = []

    TS_true = []
    TS_pred = []

    EL_true = []
    EL_pred = []

    for features, targets in test_data:
        features = features.cuda()
        targets = targets.cuda()
        test_model = test_model.cuda()
        outputs = test_model(features)
        YS_true.append(targets.cpu().detach().numpy()[0][0][0])
        YS_pred.append(outputs.cpu().detach().numpy()[0][0][0])
        TS_true.append(targets.cpu().detach().numpy()[0][0][1])
        TS_pred.append(outputs.cpu().detach().numpy()[0][0][1])
        EL_true.append(targets.cpu().detach().numpy()[0][0][2])
        EL_pred.append(outputs.cpu().detach().numpy()[0][0][2])
    #计算YS的RMSE,MAPE,R2
    YS_true = np.array(YS_true)
    YS_pred = np.array(YS_pred)
    YS_MSE = mean_squared_error(YS_true, YS_pred)
    YS_RMSE = np.sqrt(YS_MSE)
    YS_MAPE = np.mean(np.abs((YS_pred - YS_true) / YS_true))
    YS_R2 = 1 - YS_MSE / np.var(YS_true)
    print(f'YS_RMSE: {YS_RMSE:.4f},YS_MAPE: {YS_MAPE:.4f},YS_R2: {YS_R2:.4f}')
    #计算TS的RMSE,MAPE,R2
    TS_true = np.array(TS_true)
    TS_pred = np.array(TS_pred)
    TS_MSE = mean_squared_error(TS_true, TS_pred)
    TS_RMSE = np.sqrt(TS_MSE)
    TS_MAPE = np.mean(np.abs((TS_pred - TS_true) / TS_true))
    TS_R2 = 1 - TS_MSE / np.var(TS_true)
    print(f'TS_RMSE: {TS_RMSE:.4f},TS_MAPE: {TS_MAPE:.4f},TS_R2: {TS_R2:.4f}')
    #计算EL的RMSE,MAPE,R2
    EL_true = np.array(EL_true)
    EL_pred = np.array(EL_pred)
    EL_MSE = mean_squared_error(EL_true, EL_pred)
    EL_RMSE = np.sqrt(EL_MSE)
    EL_MAPE = np.mean(np.abs((EL_pred - EL_true) / EL_true))
    EL_R2 = 1 - EL_MSE / np.var(EL_true)
    print(f'EL_RMSE: {EL_RMSE:.4f},EL_MAPE: {EL_MAPE:.4f},EL_R2: {EL_R2:.4f}')
    

    return YS_pred, TS_pred, EL_pred, YS_true, TS_true, EL_true

YS_pred, TS_pred, EL_pred, YS_true, TS_true, EL_true = test(model, test_data)




YS_RMSE: 8.8894,YS_MAPE: 0.0235,YS_R2: 0.9935
TS_RMSE: 9.2999,TS_MAPE: 0.0145,TS_R2: 0.9963
EL_RMSE: 1.8741,EL_MAPE: 0.0395,EL_R2: 0.9611
